## CNN 

In [1]:
from PyFunctions import Functions as func
from PyFunctions import var
from PyFunctions import ModelFunc as modelfunc
import os
from tqdm import tqdm
import numpy as np
import cv2

Using plaidml.keras.backend backend.


### Images

- Within the folder tests, add whatever image you would like that contains a gun and run the cell below

In [2]:
def get_img_prediction_bounding_box(path, model, dim, edge = False):
    '''This function will create a bounding box over what it believes is a weapon given the image path, dimensions, and model used to detect the weapon.  Dimensions can be found within the Var.py file.  This function is still being used as I need to apply non-max suppresion to create only one bounding box'''
    img = func.get_image_value(path, dim, edge = edge)

    if edge == True:
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
    else: 
        img = img.reshape(1, img.shape[0], img.shape[1], 3)
    
    pred = model.predict(img)[0]
    
    category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
    
    print(f'{path}\t\tPrediction: {cat}\t{round(pred.max()*100)}% Confident')
    
    clone = img.copy() 
    clone2 = img.copy()
    
    if cat_index != 0:
        img = cv2.imread(path)
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchQuality()
    #     ss.switchToSelectiveSearchFast()

        rects = ss.process() 

        windows = []
        locations = []
        print(f'Creating Bounding Boxes for {path}')
        for x, y, w,h in rects: 
            startx = x 
            starty = y 
            endx = x+w 
            endy = y+h 
            roi = img[starty:endy, startx:endx]
            if edge == True:
                roi = get_edged(roi, dim = dim)
            roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
            windows.append(roi)
            locations.append((startx, starty, endx, endy))

        windows = np.array(windows)
        if edge == True:
            windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
        else: 
            windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
        windows = np.array(windows)
        locations = np.array(locations)
        predictions = model.predict(windows)

        cat_predictions = predictions[:,cat_index]
        pred_max_idx = np.argmax(cat_predictions)
        pred_max = cat_predictions[pred_max_idx]

        pred_max_window = locations[pred_max_idx]
        startx, starty, endx, endy = pred_max_window
        cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)


#     pick = func.non_max_suppression(locations, probs = None)

#     for idx in pick: 
#         startx, startx, endx, endy = locations[idx]
#         cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
        
    
#GOES HERE 
    cv2.imshow(f'Test', np.hstack([clone, clone2]))
    cv2.waitKey(0)
    ss.clear()


    return predictions

In [3]:
#Mobilenet MODEL
edge = False
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
mobilenet = modelfunc.get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/Mobilenet.h5') 

test_folder = 'Tests/Photos'

for i in os.listdir(test_folder):
    if i == 'ipynb_checkpoints': 
        continue
    img_path = f'{test_folder}/{i}'
    predictions = get_img_prediction_bounding_box(img_path, mobilenet, dim = var.mobilenet_dimension, edge = edge)
    

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"
C:\Users\Heeeb\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Tests/Photos/AR.jpg		Prediction: Rifle		0.9998019337654114		[1.4862098e-04 4.9498449e-05 9.9980193e-01]
Creating Bounding Boxes for Tests/Photos/AR.jpg
Tests/Photos/AR2.jpg		Prediction: Rifle		0.951600968837738		[0.00314434 0.04525468 0.95160097]
Creating Bounding Boxes for Tests/Photos/AR2.jpg
Tests/Photos/NoWeap.jpg		Prediction: No Weapon		0.9318897724151611		[0.9318898  0.04441858 0.02369164]
Creating Bounding Boxes for Tests/Photos/NoWeap.jpg
Tests/Photos/Pistol.jpg		Prediction: Handgun		0.9985342025756836		[1.7059132e-04 9.9853420e-01 1.2952359e-03]
Creating Bounding Boxes for Tests/Photos/Pistol.jpg
Tests/Photos/Pistol2.jpg		Prediction: Handgun		0.9953504204750061		[0.001173   0.9953504  0.00347659]
Creating Bounding Boxes for Tests/Photos/Pistol2.jpg
Tests/Photos/Pistol3.jpg		Prediction: Handgun		0.981045126914978		[0.00139262 0.9810451  0.01756224]
Creating Bounding Boxes for Tests/Photos/Pistol3.jpg


In [4]:
print(predictions[:,2].max())

0.662963


In [5]:
assert False

AssertionError: 

In [ ]:
#NORMAL MODEL
edge = False
dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = modelfunc.get_conv_model(dim)
normal_model.load_weights('ModelWeights/V2_NoEdge_NoAugmentation.h5') #path to the model weights

test_folder = 'Tests/Photos'

for i in os.listdir(test_folder):
    if i == 'ipynb_checkpoints': 
        continue
    img_path = f'{test_folder}/{i}'
    predictions = get_img_prediction_bounding_box(img_path, normal_model, dim = var.norm_dimension, edge = edge)

### Video 
- within the folder Tests, add whatever video (.mp4) that contains a gun and run the cell below
- **Note**: Stick t video shorter than a minute or this process will take a very long time depending on your computer

In [ ]:
edge= True 
if edge == True: 
    dim = (var.norm_dimension[0],var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('ModelWeights/CNN-ModelCheckpointWeightsNoAugment.h5')

images = get_vid_frames('../Tests/Videos/Pistol2.mp4', normal_model, var.norm_dimension, edge = True)

In [ ]:
pbar = tqdm(images, desc= 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While')
windows_prob = [window_prob_func(img, normal_model, var.norm_dimension, edge = edge) for img in pbar]

In [ ]:
vid_dim = (224,224)
vid_dim = var.norm_dimension
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Tests/VideoOutput/BoundingBox.mp4',fourcc, 10, vid_dim) #change the filename to whatever you would like


for prob, img in zip(windows_prob, images): 
    clone = img.copy()
    if prob == None:
        out.write(clone)
    else:
        (p, (startx, starty, endx, endy)) = prob
        cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
        out.write(clone)
        
out.release()
cv2.destroyAllWindows()